# 📊 Sales Dashboard – Data Preparation

**Name:** Ashna Imtiaz  

**Organization:** DevelopersHub Corporation

**Internship:** Data Science & Analytics

**Project:** Interactive Business Dashboard

**Date:** 28 August 2025  


---

## Objective
This notebook focuses on **preparing sales data for dashboarding** to develop an interactive dashboard for analyzing sales, profit, and segment-wise performance.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
df = pd.read_csv("global-super-store-dataset/Global_Superstore2.csv", encoding='ISO-8859-1')
df.head(3)

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,City,State,...,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit,Shipping Cost,Order Priority
0,32298,CA-2012-124891,31-07-2012,31-07-2012,Same Day,RH-19495,Rick Hansen,Consumer,New York City,New York,...,TEC-AC-10003033,Technology,Accessories,Plantronics CS510 - Over-the-Head monaural Wir...,2309.650,7,0.0,762.1845,933.57,Critical
1,26341,IN-2013-77878,05-02-2013,07-02-2013,Second Class,JR-16210,Justin Ritter,Corporate,Wollongong,New South Wales,...,FUR-CH-10003950,Furniture,Chairs,"Novimex Executive Leather Armchair, Black",3709.395,9,0.1,-288.7650,923.63,Critical
2,25330,IN-2013-71249,17-10-2013,18-10-2013,First Class,CR-12730,Craig Reiter,Consumer,Brisbane,Queensland,...,TEC-PH-10004664,Technology,Phones,"Nokia Smart Phone, with Caller ID",5175.171,9,0.1,919.9710,915.49,Medium


# Exploratory Data Analysis

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51290 entries, 0 to 51289
Data columns (total 24 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Row ID          51290 non-null  int64  
 1   Order ID        51290 non-null  object 
 2   Order Date      51290 non-null  object 
 3   Ship Date       51290 non-null  object 
 4   Ship Mode       51290 non-null  object 
 5   Customer ID     51290 non-null  object 
 6   Customer Name   51290 non-null  object 
 7   Segment         51290 non-null  object 
 8   City            51290 non-null  object 
 9   State           51290 non-null  object 
 10  Country         51290 non-null  object 
 11  Postal Code     9994 non-null   float64
 12  Market          51290 non-null  object 
 13  Region          51290 non-null  object 
 14  Product ID      51290 non-null  object 
 15  Category        51290 non-null  object 
 16  Sub-Category    51290 non-null  object 
 17  Product Name    51290 non-null 

In [ ]:
df.describe()

,Row ID,Postal Code,Sales,Quantity,Discount,Profit,Shipping Cost
count,51290.00000,9994.000000,51290.000000,51290.000000,51290.000000,51290.000000,51290.000000
mean,25645.50000,55190.379428,246.490581,3.476545,0.142908,28.610982,26.375915
std,14806.29199,32063.693350,487.565361,2.278766,0.212280,174.340972,57.296804
min,1.00000,1040.000000,0.444000,1.000000,0.000000,-6599.978000,0.000000
25%,12823.25000,23223.000000,30.758625,2.000000,0.000000,0.000000,2.610000
50%,25645.50000,56430.500000,85.053000,3.000000,0.000000,9.240000,7.790000
75%,38467.75000,90008.000000,251.053200,5.000000,0.200000,36.810000,24.450000
max,51290.00000,99301.000000,22638.480000,14.000000,0.850000,8399.976000,933.570000


# Feature Engineering

In [ ]:
# Select only relevant columns
selected_cols = [
    'Order Date', 'Customer Name', 'Segment', 'Region',
    'Category', 'Sub-Category', 'Sales', 'Profit', 'Quantity', 'Discount'
]
df = df[selected_cols]

### Correct feature data types

In [ ]:
# Convert Order Date and Ship Date to datetime
df['Order Date'] = pd.to_datetime(df['Order Date'])\

# Ensure numeric columns are correct
df['Sales'] = pd.to_numeric(df['Sales'], errors='coerce')
df['Profit'] = pd.to_numeric(df['Profit'], errors='coerce')


In [ ]:
# Example: Profit Margin
df['Profit Margin'] = df['Profit'] / df['Sales']

# Extract Year & Month for trend analysis
df['Year'] = df['Order Date'].dt.year
df['Month'] = df['Order Date'].dt.month_name()


In [ ]:
# Optional: Aggregate data for KPIs or charts
# Example: Sales and Profit by Region
agg_region = df.groupby('Region')[['Sales', 'Profit']].sum().reset_index()

In [ ]:
# Example: Top 5 Customers by Sales
top_customers = df.groupby('Customer Name')['Sales'].sum().sort_values(ascending=False).head(5).reset_index()


In [ ]:
top_customers

,Customer Name,Sales
0,Tom Ashbrook,40488.07080
1,Tamara Chand,37457.33300
2,Greg Tran,35550.95428
3,Christopher Conant,35187.07640
4,Sean Miller,35170.93296


In [ ]:
df

,Order Date,Customer Name,Segment,Region,Category,Sub-Category,Sales,Profit,Quantity,Discount,Profit Margin,Year,Month
0,2012-07-31,Rick Hansen,Consumer,East,Technology,Accessories,2309.650,762.1845,7,0.0,0.330000,2012,July
1,2013-02-05,Justin Ritter,Corporate,Oceania,Furniture,Chairs,3709.395,-288.7650,9,0.1,-0.077847,2013,February
2,2013-10-17,Craig Reiter,Consumer,Oceania,Technology,Phones,5175.171,919.9710,9,0.1,0.177766,2013,October
3,2013-01-28,Katherine Murray,Home Office,Central,Technology,Phones,2892.510,-96.5400,5,0.1,-0.033376,2013,January
4,2013-11-05,Rick Hansen,Consumer,Africa,Technology,Copiers,2832.960,311.5200,8,0.0,0.109963,2013,November
...,...,...,...,...,...,...,...,...,...,...,...,...,...
51285,2014-06-19,Katrina Edelman,Corporate,North Asia,Office Supplies,Fasteners,65.100,4.5000,5,0.0,0.069124,2014,June
51286,2014-06-20,Zuschuss Carroll,Consumer,Central,Office Supplies,Appliances,0.444,-1.1100,1,0.8,-2.500000,2014,June
51287,2013-12-02,Laurel Beltran,Home Office,West,Office Supplies,Envelopes,22.920,11.2308,3,0.0,0.490000,2013,December
51288,2012-02-18,Ross Baird,Home Office,South,Office Supplies,Binders,13.440,2.4000,2,0.0,0.178571,2012,February


In [ ]:
# Save cleaned dataset for dashboard use
df.to_csv("dashboard_ready.csv", index=False)
print("✅ Cleaned dataset saved as dashboard_ready.csv")

✅ Cleaned dataset saved as dashboard_ready.csv
